In [16]:
import config_readwrite as crw
import glob
import numpy as np
import os, sys
import pandas as pd
import pybedtools as pbt

from scipy import stats

In [17]:
# biomarin MPRA
config, cfn= crw.read("/wynton/home/ahituv/fongsl/biomarin/bin/config.neuron.mpra.orig.ini")
MPRA = config["lib2"]['delta.mpra_centeredz'] #["enh_bed"] #["sig-wilcoxon"] reps l2(rna/dna)
ENH_BED = config["lib2"]['enh_bed']#["sig-wilcoxon"] reps l2(rna/dna)

MPRA_BED_NONA = ENH_BED.strip(".bed") + ".genomic_only.bed"
MPRA_BED_x_SONG = ENH_BED.strip(".bed") + ".x.SONG2020.bed"

MPRA_DATAPATH = "/wynton/group/ahituv/fongsl/projects/biomarin/data"
config["lib2"]['enh_bed_genomic']=MPRA_BED_NONA

SUPPORT = config["lib2"]["support"] # neighboring tile support

# Song data
DATA_PATH = "/wynton/group/ahituv/data/encode/shen_bdbag_2019/data/"
os.chdir(DATA_PATH) # go to plac-seq path

# write
combined = os.path.join(DATA_PATH, f"iN.eN.IPC.RG.HiC.bed")  # combine all hic coordinates together
collapsed = os.path.join(DATA_PATH, f"iN.eN.IPC.RG.HiC.collapsed.bed")  # collapse all hic annotation 
collapsed_cl = os.path.join(DATA_PATH, f"iN.eN.IPC.RG.HiC.collapsed.celltype.only.bed")  # collapse all hic annotation 

SONG_ATAC_PATH = "/wynton/group/ahituv/data/encode/shen_bdbag_2019/data/embargo/shen/accessible_chromatin/processed/peaks"
EN_ATAC = os.path.join(SONG_ATAC_PATH, "eN.ATAC-seq.narrowPeak")
IN_ATAC = os.path.join(SONG_ATAC_PATH, "iN.ATAC-seq.narrowPeak")

MPRA_BED_x_SONGEN = os.path.join(MPRA_DATAPATH,"eN.ATAC-seq.narrowPeak" + ".x.MPRA.bed")
MPRA_BED_x_SONGIN = os.path.join(MPRA_DATAPATH,"iN.ATAC-seq.narrowPeak" + ".x.MPRA.bed")

section = "song_2020_placseq"
#crw.check(section, config)

config[section]["path"] = DATA_PATH
config[section]["combined"] = combined
config[section]["collapsed"] = collapsed
config[section]["collapsed_cl"] = collapsed_cl
config[section]["mpra_bed_x_song"] = MPRA_BED_x_SONG
config[section]["atac_path"] = SONG_ATAC_PATH
config[section]["atac_in"] = IN_ATAC
config[section]["atac_en"] = EN_ATAC
config[section]["mpra_bed_x_songinATAC"] = MPRA_BED_x_SONGIN
config[section]["mpra_bed_x_songenATAC"] = MPRA_BED_x_SONGEN



crw.write(config, cfn)

# Main

## load plac

In [3]:
plac = pd.read_csv(combined, sep='\t')
plac

/scratch/fongsl/ipykernel_694520/2542846049.py:1: DtypeWarning: Columns (1,2,3,4,5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  plac = pd.read_csv(combined, sep='\t')


,#chr,start,end,count,expected,fdr,celltype,id,link,link_id
0,chr1,890000,895000,36,13.7267118749079,7.08770821896245e-05,iN,iN.14,chr1:890000-895000_chr1:900000-905000,1
1,chr1,900000,905000,36,13.7267118749079,7.08770821896245e-05,iN,iN.14,chr1:890000-895000_chr1:900000-905000,2
2,chr1,1000000,1005000,13,3.17293864858796,0.0005259819865644,iN,iN.259,chr1:1000000-1005000_chr1:1300000-1305000,1
3,chr1,1005000,1010000,13,3.19541920865153,0.0015132910675641,iN,iN.1938,chr1:1005000-1010000_chr1:1305000-1310000,1
4,chr1,1020000,1025000,17,3.87774128536577,2.41664334727065e-05,iN,iN.218,chr1:1020000-1025000_chr1:1320000-1325000,1
...,...,...,...,...,...,...,...,...,...,...
226782,chr9,137450000,137455000,13,2.981913,0.000476,RG,RG.17410,chr9:137290000-137295000_chr9:137450000-137455000,2
226783,chr9,137465000,137470000,17,3.509233,0.000011,RG,RG.17351,chr9:137290000-137295000_chr9:137465000-137470000,2
226784,chr9,137535000,137540000,14,2.401291,0.000013,RG,RG.15806,chr9:137045000-137050000_chr9:137535000-137540000,2
226785,chr9,137560000,137565000,47,18.546754,0.000003,RG,RG.15813,chr9:137560000-137565000_chr9:137575000-137580000,1


In [4]:
### drop any na columns (positive controls, negative controls, synthetics, shuffled)

lib = pd.read_csv(ENH_BED, sep='\t')

print(lib.shape, lib.dropna().shape)
list(set(lib.loc[lib["enh.name"].isna(), "name"]))[:3]

(44044, 11) (42366, 11)


['Backgroundseq2chr1:244100624-244100893:+|Pos1:115,Pos2:155|Motiffamily:glutmotif2,Motiffamily:glutmotif1|Motif1:TTCCTCATATGT|Motif2:TGAACTGCTAGA',
 'Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Motiffamily:glutmotif2,Motiffamily:glutmotif1|Motif1:TTCATTATGTGT|Motif2:TGAACTGCTAGA',
 'Backgroundseq2chr1:244100624-244100893:+|Pos1:115,Pos2:155|Motiffamily:glutmotif2,Motiffamily:glutmotif2|Motif1:TTCATCATGTGT|Motif2:TTCCTCATATGT']

In [5]:
lib.loc[lib["enh.name"]=="enh.1530"]

,#chr,start.tile,end.tile,coor,#chr.enh,start.enh,end.enh,enh.id,enh.name,tile.order,name
37062,chr6,109381579.0,109381849.0,chr6:109381579-109381849,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,0.0,chr6:109381579-109381849
37063,chr6,109381599.0,109381869.0,chr6:109381599-109381869,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,1.0,chr6:109381599-109381869
37064,chr6,109381619.0,109381889.0,chr6:109381619-109381889,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,2.0,chr6:109381619-109381889
37065,chr6,109381639.0,109381909.0,chr6:109381639-109381909,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,3.0,chr6:109381639-109381909
37066,chr6,109381659.0,109381929.0,chr6:109381659-109381929,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,4.0,chr6:109381659-109381929
37067,chr6,109381679.0,109381949.0,chr6:109381679-109381949,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,5.0,chr6:109381679-109381949
37068,chr6,109381699.0,109381969.0,chr6:109381699-109381969,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,6.0,chr6:109381699-109381969
37069,chr6,109381719.0,109381989.0,chr6:109381719-109381989,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,7.0,chr6:109381719-109381989
37070,chr6,109381739.0,109382009.0,chr6:109381739-109382009,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,8.0,chr6:109381739-109382009


In [6]:
# if os.path.exists(MPRA_BED_NONA) is False:
genome_only = lib.loc[~lib["enh.name"].isna()].copy()
genome_only[genome_only.columns[1:3]] = genome_only[genome_only.columns[1:3]].astype(int)
genome_only.to_csv(MPRA_BED_NONA, sep='\t', index=False)  # write

## intersect mpra x song_2020

In [7]:
mprabed = pbt.BedTool(MPRA_BED_NONA)

#@if os.path.exists(MPRA_BED_x_SONG) is False:
mprabed = mprabed.sort()
song2020 = pbt.BedTool(collapsed_cl).sort()

mprabed.intersect(song2020, wa=True, wb=True, output=MPRA_BED_x_SONG)


song2020_atac_in = pbt.BedTool(IN_ATAC).sort()
song2020_atac_en = pbt.BedTool(EN_ATAC).sort()

mprabed.intersect(song2020_atac_en, wa=True, wb=True, output=MPRA_BED_x_SONGEN)

mprabed.intersect(song2020_atac_in, wa=True, wb=True, output=MPRA_BED_x_SONGIN)


<BedTool(/wynton/group/ahituv/fongsl/projects/biomarin/data/iN.ATAC-seq.narrowPeak.x.MPRA.bed)>

# load song intersection only

## atac peaks

In [19]:
cols = ['#chr',
        'start.tile',
        'end.tile',
        'coor',
        '#chr.enh',
        'start.enh',
        'end.enh',
        'enh.id',
        'enh.name',
        'tile.order',
        'name', "#chr_song", "start_song", "end_song", "peak"]
mpra_song_en = pd.read_csv(MPRA_BED_x_SONGEN, sep='\t', header=None, usecols=np.arange(0, 15))
mpra_song_en.columns=cols # columns
mpra_song_en["peak_eN"]="eN." + mpra_song_en["peak"] # anotate peak
mpra_song_en.loc[mpra_song_en["enh.name"]=="enh.123"]

,#chr,start.tile,end.tile,coor,#chr.enh,start.enh,end.enh,enh.id,enh.name,tile.order,name,#chr_song,start_song,end_song,peak,peak_eN
2120,chr1,180501931,180502201,chr1:180501931-180502201,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,0.0,chr1:180501931-180502201,chr1,180502140,180503392,Peak_8625,eN.Peak_8625
2121,chr1,180501951,180502221,chr1:180501951-180502221,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,1.0,chr1:180501951-180502221,chr1,180502140,180503392,Peak_8625,eN.Peak_8625
2122,chr1,180501971,180502241,chr1:180501971-180502241,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,2.0,chr1:180501971-180502241,chr1,180502140,180503392,Peak_8625,eN.Peak_8625
2123,chr1,180501991,180502261,chr1:180501991-180502261,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,3.0,chr1:180501991-180502261,chr1,180502140,180503392,Peak_8625,eN.Peak_8625
2124,chr1,180502003,180502273,chr1:180502003-180502273,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,4.0,chr1:180502003-180502273,chr1,180502140,180503392,Peak_8625,eN.Peak_8625
2125,chr1,180502011,180502281,chr1:180502011-180502281,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,5.0,chr1:180502011-180502281,chr1,180502140,180503392,Peak_8625,eN.Peak_8625
2126,chr1,180502023,180502293,chr1:180502023-180502293,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,6.0,chr1:180502023-180502293,chr1,180502140,180503392,Peak_8625,eN.Peak_8625
2127,chr1,180502031,180502301,chr1:180502031-180502301,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,7.0,chr1:180502031-180502301,chr1,180502140,180503392,Peak_8625,eN.Peak_8625
2128,chr1,180502043,180502313,chr1:180502043-180502313,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,8.0,chr1:180502043-180502313,chr1,180502140,180503392,Peak_8625,eN.Peak_8625
2129,chr1,180502051,180502321,chr1:180502051-180502321,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,9.0,chr1:180502051-180502321,chr1,180502140,180503392,Peak_8625,eN.Peak_8625


In [9]:
mpra_song_in = pd.read_csv(MPRA_BED_x_SONGIN, sep='\t', header=None, usecols=np.arange(0, 15))
mpra_song_in.columns=cols
mpra_song_in["peak_iN"]="iN." + mpra_song_in["peak"]
mpra_song_in.loc[mpra_song_in["enh.name"]=="enh.1530"]

,#chr,start.tile,end.tile,coor,#chr.enh,start.enh,end.enh,enh.id,enh.name,tile.order,name,#chr_song,start_song,end_song,peak,peak_iN
19434,chr6,109381719,109381989,chr6:109381719-109381989,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,7.0,chr6:109381719-109381989,chr6,109381972,109383471,Peak_1804,iN.Peak_1804
19435,chr6,109381739,109382009,chr6:109381739-109382009,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,8.0,chr6:109381739-109382009,chr6,109381972,109383471,Peak_1804,iN.Peak_1804


## hic

In [10]:
cols = ['#chr',
        'start.tile',
        'end.tile',
        'coor',
        '#chr.enh',
        'start.enh',
        'end.enh',
        'enh.id',
        'enh.name',
        'tile.order',
        'name', "#chr_song", "start_song", "end_song", "celltype"]

mpra_song = pd.read_csv(MPRA_BED_x_SONG, sep='\t', header=None, names=cols)
mpra_song_en = pd.read_csv(MPRA_BED_x_SONGEN, sep='\t', header=None, names=cols)
mpra_song.head()

,#chr,start.tile,end.tile,coor,#chr.enh,start.enh,end.enh,enh.id,enh.name,tile.order,name,#chr_song,start_song,end_song,celltype
0,chr1,2440188,2440458,chr1:2440188-2440458,chr1,2440188.0,2440764.0,chr1:2440188-2440764,enh.3,0.0,chr1:2440188-2440458,chr1,2440000,2445000,eN
1,chr1,2440208,2440478,chr1:2440208-2440478,chr1,2440188.0,2440764.0,chr1:2440188-2440764,enh.3,1.0,chr1:2440208-2440478,chr1,2440000,2445000,eN
2,chr1,2440228,2440498,chr1:2440228-2440498,chr1,2440188.0,2440764.0,chr1:2440188-2440764,enh.3,2.0,chr1:2440228-2440498,chr1,2440000,2445000,eN
3,chr1,2440248,2440518,chr1:2440248-2440518,chr1,2440188.0,2440764.0,chr1:2440188-2440764,enh.3,3.0,chr1:2440248-2440518,chr1,2440000,2445000,eN
4,chr1,2440259,2440529,chr1:2440259-2440529,chr1,2440188.0,2440764.0,chr1:2440188-2440764,enh.3,4.0,chr1:2440259-2440529,chr1,2440000,2445000,eN


In [21]:
mpra_song.loc[mpra_song["enh.name"]=="enh.123"]

,#chr,start.tile,end.tile,coor,#chr.enh,start.enh,end.enh,enh.id,enh.name,tile.order,name,#chr_song,start_song,end_song,celltype
1471,chr1,180501931,180502201,chr1:180501931-180502201,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,0.0,chr1:180501931-180502201,chr1,180500000,180505000,"IPC,RG,eN,iN"
1472,chr1,180501951,180502221,chr1:180501951-180502221,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,1.0,chr1:180501951-180502221,chr1,180500000,180505000,"IPC,RG,eN,iN"
1473,chr1,180501971,180502241,chr1:180501971-180502241,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,2.0,chr1:180501971-180502241,chr1,180500000,180505000,"IPC,RG,eN,iN"
1474,chr1,180501991,180502261,chr1:180501991-180502261,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,3.0,chr1:180501991-180502261,chr1,180500000,180505000,"IPC,RG,eN,iN"
1475,chr1,180502003,180502273,chr1:180502003-180502273,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,4.0,chr1:180502003-180502273,chr1,180500000,180505000,"IPC,RG,eN,iN"
1476,chr1,180502011,180502281,chr1:180502011-180502281,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,5.0,chr1:180502011-180502281,chr1,180500000,180505000,"IPC,RG,eN,iN"
1477,chr1,180502023,180502293,chr1:180502023-180502293,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,6.0,chr1:180502023-180502293,chr1,180500000,180505000,"IPC,RG,eN,iN"
1478,chr1,180502031,180502301,chr1:180502031-180502301,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,7.0,chr1:180502031-180502301,chr1,180500000,180505000,"IPC,RG,eN,iN"
1479,chr1,180502043,180502313,chr1:180502043-180502313,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,8.0,chr1:180502043-180502313,chr1,180500000,180505000,"IPC,RG,eN,iN"
1480,chr1,180502051,180502321,chr1:180502051-180502321,chr1,180501931.0,180502524.0,chr1:180501931-180502524,enh.123,9.0,chr1:180502051-180502321,chr1,180500000,180505000,"IPC,RG,eN,iN"


## MPRA data

In [22]:
mpra = pd.read_csv(MPRA, sep='\t')

mpra = pd.merge(mpra, lib[['enh.name', 'tile.order', 'name']], how="left").drop_duplicates()  # no shuffles, positive controls, neg controls 
mpra.head()

,name,gaba-label,glut-label,sig,celltype_dif,gaba,glut,delta.gaba-glut,fdr,-log10p_fdr,bs,enh.name,tile.order
0,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",silencing,False,False,True,-2.699699,-1.696544,-1.003154,0.119390,0.923032,False,NaN,NaN
1,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",silencing,False,False,True,-2.683608,-1.740263,-0.943346,0.493535,0.306682,False,NaN,NaN
2,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",False,False,False,False,-2.228061,-1.842682,-0.385379,0.728743,0.137425,False,NaN,NaN
3,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",silencing,False,False,True,-2.666574,-1.554678,-1.111896,0.364764,0.437989,False,NaN,NaN
4,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",silencing,silencing,False,False,-2.756752,-2.505154,-0.251599,0.801537,0.096076,False,NaN,NaN


## support data

In [23]:
support = pd.read_csv(SUPPORT, sep='\t')

# enh_ids with neighbor support
supported_enhid = support.loc[support["neighbor_either_label"]!=False, "enh.name"].unique()
print(len(supported_enhid))

# merge support with mpra 
mpra = pd.merge(mpra, support, how="left").drop_duplicates()

support

1740


,name,tile.order,support_dif,support_dif_code,support_dif_name,neighbor_dif,neighbor_gaba_label,neighbor_glut_label,enh.name,neighbor_either_label
0,chr15:58748908-58749178,0.0,0,0,NaN,NaN,NaN,NaN,enh.566,None.None
1,chr15:58748928-58749198,1.0,0,0,NaN,NaN,NaN,NaN,enh.566,None.None
2,chr15:58748948-58749218,2.0,0,0,NaN,NaN,NaN,NaN,enh.566,None.None
3,chr15:58748968-58749238,3.0,0,0,NaN,NaN,NaN,NaN,enh.566,None.None
4,chr15:58748988-58749258,4.0,0,0,NaN,NaN,NaN,NaN,enh.566,None.None
...,...,...,...,...,...,...,...,...,...,...
42171,chr19:44891616-44891886,134.0,0,0,NaN,NaN,NaN,NaN,enh.810,None.None
42172,chr19:44891636-44891906,135.0,0,0,NaN,NaN,NaN,NaN,enh.810,None.None
42173,chr19:44891656-44891926,136.0,0,0,NaN,NaN,NaN,NaN,enh.810,None.None
42174,chr19:44891676-44891946,137.0,1,10,cat-only,NaN,NaN,NaN,enh.810,None.None


In [24]:
len(set(mpra["enh.name"]))

1699

## merge MPRA w hic

In [25]:
int = pd.merge(mpra, mpra_song, how="left").drop_duplicates()
int = pd.merge(int, mpra_song_en[["enh.name","peak_eN"]], how="left").drop_duplicates()
int = pd.merge(int, mpra_song_in[["enh.name","peak_iN"]], how="left").drop_duplicates()

drop_ind = int.loc[(int["name"].str.contains("shuf"))|
(int["name"].str.contains("seq"))|
(int["name"].str.contains("top"))|
(int["name"].str.contains("bottom"))
].index


int = int.loc[~int.index.isin(drop_ind)]
int.head()

,name,gaba-label,glut-label,sig,celltype_dif,gaba,glut,delta.gaba-glut,fdr,-log10p_fdr,...,#chr.enh,start.enh,end.enh,enh.id,#chr_song,start_song,end_song,celltype,peak_eN,peak_iN
974,Negative-GABA_chr10:26471819-26472088:+,False,False,False,False,-1.614119,-1.590097,-0.024022,0.924293,0.034190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
975,Negative-GABA_chr10:26798964-26799233:-,False,False,False,False,-1.785743,-1.543796,-0.241947,0.138986,0.857028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
976,Negative-GABA_chr10:26799114-26799383:-,False,False,False,False,-1.425355,-1.231964,-0.193391,0.662463,0.178838,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
977,Negative-GABA_chr10:26799129-26799398:-,False,False,False,False,-1.444640,-1.449871,0.005231,0.985000,0.006564,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
978,Negative-GABA_chr10:26799159-26799428:-,False,False,False,False,-1.590640,-1.124827,-0.465813,0.384536,0.415063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
int.loc[int["enh.name"]=="enh.1530"]

,name,gaba-label,glut-label,sig,celltype_dif,gaba,glut,delta.gaba-glut,fdr,-log10p_fdr,...,#chr.enh,start.enh,end.enh,enh.id,#chr_song,start_song,end_song,celltype,peak_eN,peak_iN
906998,chr6:109381579-109381849,False,False,False,False,-0.573416,-0.893999,0.320583,0.413896,0.383108,...,chr6,109381579.0,109382009.0,chr6:109381579-109382009,chr6,109380000.0,109385000.0,"RG,eN",eN.Peak_1349,iN.Peak_1804
907000,chr6:109381599-109381869,False,False,False,False,-0.632854,-0.655745,0.022891,0.947230,0.023545,...,chr6,109381579.0,109382009.0,chr6:109381579-109382009,chr6,109380000.0,109385000.0,"RG,eN",eN.Peak_1349,iN.Peak_1804
907002,chr6:109381619-109381889,False,False,False,False,0.133846,-0.365394,0.499240,0.250056,0.601962,...,chr6,109381579.0,109382009.0,chr6:109381579-109382009,chr6,109380000.0,109385000.0,"RG,eN",eN.Peak_1349,iN.Peak_1804
907004,chr6:109381639-109381909,False,False,False,False,-0.261023,-0.300627,0.039604,0.900477,0.045528,...,chr6,109381579.0,109382009.0,chr6:109381579-109382009,chr6,109380000.0,109385000.0,"RG,eN",eN.Peak_1349,iN.Peak_1804
907006,chr6:109381659-109381929,False,False,False,False,-0.624753,0.064400,-0.689153,0.341217,0.466969,...,chr6,109381579.0,109382009.0,chr6:109381579-109382009,chr6,109380000.0,109385000.0,"RG,eN",eN.Peak_1349,iN.Peak_1804
907008,chr6:109381699-109381969,False,False,False,False,-0.242038,-0.620566,0.378528,0.401580,0.396228,...,chr6,109381579.0,109382009.0,chr6:109381579-109382009,chr6,109380000.0,109385000.0,"RG,eN",eN.Peak_1349,iN.Peak_1804
907010,chr6:109381719-109381989,False,False,False,False,-0.991293,-0.723489,-0.267804,0.256778,0.590441,...,chr6,109381579.0,109382009.0,chr6:109381579-109382009,chr6,109380000.0,109385000.0,"RG,eN",eN.Peak_1349,iN.Peak_1804
907012,chr6:109381739-109382009,False,False,False,False,-1.400453,-0.686316,-0.714137,0.182583,0.738541,...,chr6,109381579.0,109382009.0,chr6:109381579-109382009,chr6,109380000.0,109385000.0,"RG,eN",eN.Peak_1349,iN.Peak_1804


In [27]:
602/1699

0.3543260741612713

In [28]:
len(lib["enh.id"].unique()), len(int["enh.id"].unique())

(1741, 602)

In [29]:
cols = ['neighbor_either_label', 'celltype', "enh.name"]
int[cols].drop_duplicates().groupby(cols[:-1])[cols[-1]].count()

neighbor_either_label          celltype    
None.None                      IPC              23
                               IPC,RG           33
                               IPC,RG,eN        34
                               IPC,RG,eN,iN    155
                               IPC,RG,iN        28
                                              ... 
enh.888.gaba.1.enh.888.glut.1  IPC,RG,eN,iN      1
enh.914.gaba.0.None            IPC,RG,iN         1
enh.92.gaba.0.enh.92.glut.0    RG,iN             1
enh.924.gaba.0.enh.924.glut.0  iN                1
enh.966.gaba.0.None            RG,iN             1
Name: enh.name, Length: 405, dtype: int64

# FET - active element in song cell type? 

    # test 1 - MPRA active enhancer enriched for plac-seq?
                active either | not active either
        Song+
    not Song+

    # test 2 - MPRA active tile label enriched for plac-seq?
                active either tile label | not active either tile label
        Song+
    not Song+

    # test 3 - MPRA enriched for cell-type-specific plac-seq (e.g. glut v. en | gaba v. in)
                active glut | not active glut
        en+
    not en+

In [30]:
def fetPlac(active_mpra, inactive_mpra, inplac, outplac):
    # intersect elements to get counts. 
    a = len(active_mpra.intersection(inplac))
    b = len(inactive_mpra.intersection(inplac))
    c = len(active_mpra.intersection(outplac))
    d = len(inactive_mpra.intersection(outplac))
    
    contingency_table=[[a,b], [c,d]]
    
    print(contingency_table)
    OR, p = stats.fisher_exact(contingency_table)
    print(OR, p)

## test 1 - is MPRA active (w/ support) locus enriched for PLAC-seq?
    - use neighbor support criteria, perform at enhancer level

    - possible confounder: MPRA enhancer loci are different lengths, have different expectation of overlap. Could get around by using shuffling strategy? 

In [37]:
mpra_unit = "enh.name"  # measure at the locus level or the tile level ("name")

# get active MPRA enhancer loci (w neighbor support)
active_mpra = set(int.loc[int["neighbor_either_label"]!='None.None', mpra_unit])

# get inactive MPRA enhancer loci (w neighbor support)
inactive_mpra_ = set(int.loc[int["neighbor_either_label"]=='None.None', mpra_unit])

# remove any elements that have both active and inactive regions in an enhancer locus from inactive enhancer set. 
inactive_mpra = inactive_mpra_.difference(active_mpra)

# get all enhancers that overlap song elements
inplac = set(int.loc[~int["#chr_song"].isna(), mpra_unit])

# get all enhancers that do not overlap song elements
outplac_ = set(int.loc[int["#chr_song"].isna(), mpra_unit])

# remove any elements that are both in and out of plac
outplac = outplac_.difference(inplac)

fetPlac(active_mpra, inactive_mpra, inplac, outplac)

[[244, 357], [422, 674]]
1.0916138967435316 0.4059534693331285


### Results test 1 - No. enrichment of MPRA active loci with Plac-seq, though trending. 

    # test 1 - MPRA active enhancer enriched for plac-seq?
                active either | not active either
        Song+          A            B
    not Song+          C            D

    [[A,B], [C,D]]
    [[244, 357], [422, 674]]


    FET OR, P
   1.0916138967435316 0.4059534693331285

## Test 2 - is MPRA active enhancer (one active tile) enriched for PLAC-seq?
 - use tile-level inference                                     
    

In [39]:
mpra_unit = "enh.name"

# get active MPRA enhancer loci (wo neighbor support, tile support only)
active_mpra = set(int.loc[(int["gaba-label"] != "False") |
                  (int["glut-label"] != "False"), mpra_unit])

# get inactive MPRA enhancer loci (wo neighbor support, tile support only)
inactive_mpra_ = set(int.loc[(int["gaba-label"] == "False") &
                             (int["glut-label"] == "False"), mpra_unit])
# remove any elements that have both active and inactive regions
inactive_mpra = inactive_mpra_.difference(active_mpra)

# get all enhancers that overlap song elements
inplac = set(int.loc[~int["#chr_song"].isna(), mpra_unit])

# get all enhancers that don't overlap song elements
outplac_ = set(int.loc[int["#chr_song"].isna(), mpra_unit])

# remove any elements that are both in and out of plac
outplac = outplac_.difference(inplac)

fetPlac(active_mpra, inactive_mpra, inplac, outplac)

[[412, 189], [755, 341]]
0.9845614772767091 0.91280620581736


### Results test 2 - No. enrichment of MPRA active loci (one-tile definition) with Plac-seq

    # test 2 - MPRA active enhancer (one-tile active in either glut or gaba) enriched for plac-seq?
                active either | not active either
        Song+          A            B
    not Song+          C            D

    [[A,B], [C,D]]
    [[412, 189], [755, 341]]

    FET OR, P
       0.9845614772767091 0.91280620581736



## test 3 - cell-type-active elements enriched in cell-type plac?

In [40]:
cls = [("gaba", "iN"), 
      ("glut", "eN")]
mpra_unit = "enh.name"  # measure at the locus level or the tile level ("name")


### Results test3 - cell-type-active elements are not enriched for cell-type plac overlap
gaba, iN-containing annotations
     
      test 3 - MPRA enriched for cell-type-specific plac-seq (e.g. glut v. en | gaba v. in)
                active glut/gaba | not active glut/gaba
        en/in+
    not en/in+



     gaba, plac
    [[60, 541], [121, 975]]
    0.8936618750095476 0.5121526626263884
    
    glut, plac
    [[225, 376], [354, 742]]
    1.2542823656689506 0.03677774332185711
    
    gaba, iN-containing annotations
    [[32, 276], [149, 1241]]
    0.0 0.06905449642466382

    glut, eN-containing annotations
    [[127, 232], [452, 887]]
    0.8391903914590747 0.6043975831507908

In [44]:
for mpra_cl , plac_cl in cls:
    print(mpra_cl , plac_cl)
    # get active MPRA enhancer loci (w neighbor support)
    active_mpra = set(int.loc[int[f"neighbor_{mpra_cl}_label"].astype(str)!="nan", mpra_unit])
    
    # get inactive MPRA enhancer loci (w neighbor support)
    inactive_mpra_ = set(int.loc[int[f"neighbor_{mpra_cl}_label"].astype(str)== "nan", mpra_unit])
    
    # remove any elements that have both active and inactive regions in an enhancer locus from inactive enhancer set. 
    inactive_mpra = inactive_mpra_.difference(active_mpra)
    
    # get all enhancers that overlap song elements
    int["celltype"] = int["celltype"].fillna("none")

    # # TEST1 ALL PLAC
    # get all enhancers that overlap song elements
    inplac = set(int.loc[~int["#chr_song"].isna(), mpra_unit])
    
    # get all enhancers that don't overlap song elements
    outplac_ = set(int.loc[int["#chr_song"].isna(), mpra_unit])
    # remove any elements that are both in and out of plac
    outplac = outplac_.difference(inplac)

    # # TEST2 ONLYCL-matching PLAC
    inplac_cl = set(int.loc[int["celltype"].str.contains(plac_cl), mpra_unit])
    
    # get all enhancers that do not overlap song elements
    outplac_cl_ = set(int.loc[(~int["celltype"].str.contains(plac_cl)), mpra_unit])
    # remove any elements that are both in and out of plac_cl
    outplac_cl = outplac_cl_.difference(inplac_cl)

    # run FET
    fetPlac(active_mpra, inactive_mpra, inplac, outplac)

    fetPlac(active_mpra, inactive_mpra, inplac_cl, outplac_cl_)

gaba iN
[[63, 538], [124, 972]]
0.9179158172442738 0.6274681687135502
[[35, 273], [157, 1251]]
1.0215580597746203 0.9205662362411939
glut eN
[[233, 368], [373, 723]]
1.2272627928663014 0.05651143561550594
[[133, 226], [482, 882]]
1.0768736459442587 0.5774611600614606


In [45]:
233/(233+373), 63/(63+124)

(0.3844884488448845, 0.33689839572192515)

In [48]:
233+373, 63+124

(606, 187)

In [49]:
support

,name,tile.order,support_dif,support_dif_code,support_dif_name,neighbor_dif,neighbor_gaba_label,neighbor_glut_label,enh.name,neighbor_either_label
0,chr15:58748908-58749178,0.0,0,0,NaN,NaN,NaN,NaN,enh.566,None.None
1,chr15:58748928-58749198,1.0,0,0,NaN,NaN,NaN,NaN,enh.566,None.None
2,chr15:58748948-58749218,2.0,0,0,NaN,NaN,NaN,NaN,enh.566,None.None
3,chr15:58748968-58749238,3.0,0,0,NaN,NaN,NaN,NaN,enh.566,None.None
4,chr15:58748988-58749258,4.0,0,0,NaN,NaN,NaN,NaN,enh.566,None.None
...,...,...,...,...,...,...,...,...,...,...
42171,chr19:44891616-44891886,134.0,0,0,NaN,NaN,NaN,NaN,enh.810,None.None
42172,chr19:44891636-44891906,135.0,0,0,NaN,NaN,NaN,NaN,enh.810,None.None
42173,chr19:44891656-44891926,136.0,0,0,NaN,NaN,NaN,NaN,enh.810,None.None
42174,chr19:44891676-44891946,137.0,1,10,cat-only,NaN,NaN,NaN,enh.810,None.None


## differences in activity

In [50]:
cls = [("dif", "iN"), 
      ("dif", "eN")]
mpra_unit = "enh.name"  # measure at the locus level or the tile level ("name")

for mpra_cl , plac_cl in cls:
    print(mpra_cl , plac_cl)
   # get active MPRA enhancer loci (w neighbor support)
    active_mpra = set(int.loc[int[f"neighbor_{mpra_cl}"].astype(str)!="nan", mpra_unit])
    
    # get inactive MPRA enhancer loci (w neighbor support)
    inactive_mpra_ = set(int.loc[int[f"neighbor_{mpra_cl}"].astype(str)== "nan", mpra_unit])
    
    # remove any elements that have both active and inactive regions in an enhancer locus from inactive enhancer set. 
    inactive_mpra = inactive_mpra_.difference(active_mpra)
    
    # get all enhancers that overlap song elements
    int["celltype"] = int["celltype"].fillna("none")

    # # TEST ALL PLAC
    # get all enhancers that overlap song elements
    inplac = set(int.loc[~int["#chr_song"].isna(), mpra_unit])
    
    # get all enhancers that don't overlap song elements
    outplac_ = set(int.loc[int["#chr_song"].isna(), mpra_unit])
    # remove any elements that are both in and out of plac
    outplac = outplac_.difference(inplac)

    # # TEST CL-matching PLAC
    #inplac_cl = set(int.loc[int["celltype"]==plac_cl, mpra_unit])
    
    # get all enhancers that do not overlap song elements
    outplac_cl_ = set(int.loc[int["celltype"]!=plac_cl, mpra_unit])
    # remove any elements that are both in and out of plac_cl
    outplac_cl = outplac_cl_.difference(inplac_cl)

    # run FET
    fetPlac(active_mpra, inactive_mpra, inplac, outplac)

    fetPlac(active_mpra, inactive_mpra, inplac_cl, outplac_cl_)

dif iN
[[278, 323], [496, 600]]
1.0411465095376011 0.7213632387373563
[[164, 195], [763, 908]]
1.0008535806700944 1.0
dif eN
[[278, 323], [496, 600]]
1.0411465095376011 0.7213632387373563
[[164, 195], [745, 895]]
1.010359662708656 0.9533653727776326


## ATAC peaks
- are active MPRA elements enriched in exictatory or inhibitory atac peaks? 

In [55]:
set(int[f"neighbor_{mpra_cl}_label"])

{'enh.10.glut.0',
 'enh.1003.glut.0',
 'enh.1004.glut.0',
 'enh.1005.glut.0',
 'enh.1006.glut.0',
 'enh.1012.glut.0',
 'enh.1015.glut.0',
 'enh.1017.glut.0',
 'enh.1018.glut.0',
 'enh.1020.glut.0',
 'enh.1020.glut.1',
 'enh.1023.glut.0',
 'enh.1024.glut.0',
 'enh.1026.glut.0',
 'enh.1026.glut.1',
 'enh.1027.glut.0',
 'enh.1027.glut.1',
 'enh.1028.glut.0',
 'enh.1032.glut.0',
 'enh.1034.glut.0',
 'enh.1039.glut.0',
 'enh.1041.glut.0',
 'enh.1041.glut.1',
 'enh.1042.glut.0',
 'enh.1045.glut.0',
 'enh.1045.glut.1',
 'enh.1048.glut.0',
 'enh.1049.glut.0',
 'enh.1051.glut.0',
 'enh.1053.glut.0',
 'enh.1054.glut.0',
 'enh.1061.glut.0',
 'enh.1065.glut.0',
 'enh.1075.glut.0',
 'enh.1077.glut.0',
 'enh.108.glut.0',
 'enh.1080.glut.0',
 'enh.1081.glut.0',
 'enh.1083.glut.0',
 'enh.1083.glut.1',
 'enh.1086.glut.0',
 'enh.1086.glut.1',
 'enh.109.glut.0',
 'enh.1093.glut.0',
 'enh.1094.glut.0',
 'enh.1095.glut.0',
 'enh.1096.glut.0',
 'enh.1096.glut.1',
 'enh.1099.glut.0',
 'enh.1099.glut.1',
 'en

In [56]:
cls = [("gaba", "peak_iN"), 
      ("glut", "peak_eN")]
mpra_unit = "enh.name"  # measure at the locus level or the tile level ("name")

for mpra_cl , plac_cl in cls:
    print(mpra_cl , plac_cl)
    # get active MPRA enhancer loci (w neighbor support)
    active_mpra = set(int.loc[int[f"neighbor_{mpra_cl}_label"].astype(str)!="nan", mpra_unit])
    
    # get inactive MPRA enhancer loci (w neighbor support)
    inactive_mpra_ = set(int.loc[int[f"neighbor_{mpra_cl}_label"].astype(str)=="nan", mpra_unit])
    
    # remove any elements that have both active and inactive regions in an enhancer locus from inactive enhancer set. 
    inactive_mpra = inactive_mpra_.difference(active_mpra)
    
    # get all enhancers that overlap song elements
    #int["celltype"] = int["celltype"].fillna("none")

    # # TEST ALL PLAC
    # get all enhancers that overlap song elements
    inplac = set(int.loc[~int[plac_cl].isna(), mpra_unit])
    
    # get all enhancers that don't overlap song elements
    outplac_ = set(int.loc[int[plac_cl].isna(), mpra_unit])
    # remove any elements that are both in and out of plac
    outplac = outplac_.difference(inplac)


    # run FET
    fetPlac(active_mpra, inactive_mpra, inplac, outplac)

    

gaba peak_iN
[[96, 783], [91, 727]]
0.9794956001852554 0.9382008353133369
glut peak_eN
[[379, 577], [227, 514]]
1.4873071255697479 0.00012696459986662813


In [57]:
379/ (379+227), (379+227), 96/(96+91), (96+91)

(0.6254125412541254, 606, 0.5133689839572193, 187)

In [58]:
# annotate differentially labeled peaks

peaks = int[["enh.name", "peak_eN", "peak_iN"]].drop_duplicates().copy().dropna(how="all")

peaks['peak_eN_'] = peaks['peak_eN'].apply(lambda x: -1 if "Peak" in str(x) else 0) # -1 is Glut
peaks['peak_iN_'] = peaks['peak_iN'].apply(lambda x: 1 if "Peak" in str(x) else 0)  # 1 is Gaba
peaks["peak_dif"] = peaks['peak_iN_']+ peaks['peak_eN_']  # difference is -1 for glut-specific, 0 for not specific, 1 for gaba specific
peaks["peak_sum"] = peaks['peak_iN_']+ peaks['peak_eN_'].abs()  # if 2, peak is accessible in both cells. 
peaks.sort_values(by="peak_dif")

,enh.name,peak_eN,peak_iN,peak_eN_,peak_iN_,peak_dif,peak_sum
566364,enh.1100,eN.Peak_51127,NaN,-1,0,-1,1
356641,enh.595,eN.Peak_35218,NaN,-1,0,-1,1
356605,enh.612,eN.Peak_123092,NaN,-1,0,-1,1
355956,enh.591,eN.Peak_25433,NaN,-1,0,-1,1
355937,enh.590,eN.Peak_25433,NaN,-1,0,-1,1
...,...,...,...,...,...,...,...
708739,enh.1332,NaN,iN.Peak_109567,0,1,1,1
1052632,enh.1721,NaN,iN.Peak_35593,0,1,1,1
905739,enh.1524,NaN,iN.Peak_139956,0,1,1,1
1052524,enh.1720,NaN,iN.Peak_88786,0,1,1,1


In [59]:
int = pd.merge(int, peaks, how="left")

# enrichment of gaba-specific active elements in GABA-specific peaks? 
- no significant enrichment. 

In [62]:
cls = [("gaba", "peak_iN_", "glut"), 
      ("glut", "peak_eN_", 'gaba')]
mpra_unit = "enh.name"  # measure at the locus level or the tile level ("name")

for mpra_cl , plac_cl, mpra_not_cl in cls:
    print(mpra_cl , plac_cl)
   
    # get active MPRA enhancer loci (w neighbor support)
    active_mpra = set(int.loc[(int[f"neighbor_{mpra_cl}_label"].astype(str)!="nan")&
                              (int[f"neighbor_{mpra_not_cl}_label"].astype(str)=="nan"), 
                              mpra_unit])
    
    # get inactive MPRA enhancer loci (w neighbor support)
    inactive_mpra_ = set(int.loc[(int[f"neighbor_{mpra_cl}_label"].astype(str)!="nan")&
                                 (int[f"neighbor_{mpra_not_cl}_label"].astype(str)=="nan"), 
                                mpra_unit])
    
    # remove any elements that have both active and inactive regions in an enhancer locus from inactive enhancer set. 
    inactive_mpra = inactive_mpra_.difference(active_mpra)
    
    # get all enhancers that overlap song elements
    #int["celltype"] = int["celltype"].fillna("none")

    # # TEST ALL PLAC
    # get all enhancers that overlap song elements
    inplac = set(int.loc[(int[plac_cl]!=0)&
                 (int["peak_sum"]==1), mpra_unit])
    
    # get all enhancers that don't overlap song elements
    outplac_ = set(int.loc[(int[plac_cl]==0)&
                 (int["peak_sum"]==1), mpra_unit])
    # remove any elements that are both in and out of plac
    outplac = outplac_.difference(inplac)


    # run FET
    fetPlac(active_mpra, inactive_mpra, inplac, outplac)

    

gaba peak_iN_
[[10, 0], [24, 0]]
nan 1.0
glut peak_eN_
[[58, 0], [34, 0]]
nan 1.0


## BM candidates

In [63]:
int[["celltype", "enh.name"]].drop_duplicates().groupby("celltype").count()

,enh.name
celltype,
IPC,23
"IPC,RG",36
"IPC,RG,eN",34
"IPC,RG,eN,iN",159
"IPC,RG,iN",28
"IPC,eN",17
"IPC,eN,iN",16
"IPC,iN",11
RG,100


In [64]:
info_cols = [#'name',
 'gaba-label',
 'glut-label',
 'sig',
 'celltype_dif',
 '-log10p_fdr',
 'bs',
 'tile.order',
    '#chr_song',
 'start_song',
 'end_song',
 'celltype']

In [65]:
mpra.loc[mpra["enh.name"]=="enh.123"]

,name,gaba-label,glut-label,sig,celltype_dif,gaba,glut,delta.gaba-glut,fdr,-log10p_fdr,bs,enh.name,tile.order,support_dif,support_dif_code,support_dif_name,neighbor_dif,neighbor_gaba_label,neighbor_glut_label,neighbor_either_label
15265,chr1:180501931-180502201,False,False,True,False,1.126209,0.181919,0.944290,0.029536,1.529645,False,enh.123,0.0,1.0,100.0,sig-only,NaN,NaN,NaN,None.None
15266,chr1:180501951-180502221,False,False,False,False,0.838566,-0.190089,1.028656,0.098319,1.007364,False,enh.123,1.0,0.0,0.0,NaN,NaN,NaN,NaN,None.None
15267,chr1:180501971-180502241,False,False,True,False,0.754328,-0.158649,0.912977,0.031733,1.498489,False,enh.123,2.0,1.0,100.0,sig-only,NaN,NaN,NaN,None.None
15268,chr1:180501991-180502261,False,False,False,False,0.317693,-0.170169,0.487862,0.253722,0.595642,False,enh.123,3.0,0.0,0.0,NaN,NaN,NaN,NaN,None.None
15269,chr1:180502003-180502273,False,False,False,False,0.973876,-0.040562,1.014438,0.052865,1.276833,False,enh.123,4.0,0.0,0.0,NaN,NaN,NaN,NaN,None.None
15270,chr1:180502011-180502281,False,False,False,False,0.751934,-0.030729,0.782663,0.055755,1.253717,False,enh.123,5.0,0.0,0.0,NaN,NaN,NaN,NaN,None.None
15271,chr1:180502023-180502293,False,False,False,False,0.640551,0.168447,0.472104,0.098516,1.006494,False,enh.123,6.0,0.0,0.0,NaN,NaN,NaN,NaN,None.None
15272,chr1:180502031-180502301,False,False,False,False,0.831030,-0.174838,1.005868,0.080552,1.093923,False,enh.123,7.0,0.0,0.0,NaN,NaN,NaN,NaN,None.None
15273,chr1:180502043-180502313,False,False,True,False,0.725030,-0.158533,0.883563,0.022447,1.648842,False,enh.123,8.0,1.0,100.0,sig-only,NaN,NaN,NaN,None.None
15274,chr1:180502051-180502321,False,False,False,False,0.597959,-0.132732,0.730691,0.069507,1.157972,False,enh.123,9.0,0.0,0.0,NaN,NaN,NaN,NaN,None.None


In [66]:
# enh123 #slc2a1 
# enh 1530 SYNGAP1, excitatory neuron
enhs=["1157", "1154", "1628", "1503", "1531", "1530", "1528", "1159", "1153", "486", "123", "74", "43"]
for e in enhs:
    loc = int.loc[(int["enh.name"]==(f"enh.{e}")), info_cols]
    if loc.shape[0]>0:
        print(f"enh.{e}\n\n", loc, "\n\n")

enh.1157

       gaba-label glut-label    sig  celltype_dif  -log10p_fdr     bs  \
24146      False      False  False         False     1.255028  False   
24147      False      False  False         False     0.806970  False   
24148      False      False  False         False     0.998218  False   
24149      False      False   True         False     1.460266  False   
24150      False      False  False         False     0.808038  False   
24151      False      False  False         False     0.665558  False   
24152      False      False  False         False     0.985447  False   
24153      False      False  False         False     0.952187  False   
24154      False      False  False         False     1.161807  False   

       tile.order #chr_song  start_song  end_song celltype  
24146         0.0       NaN         NaN       NaN     none  
24147         1.0       NaN         NaN       NaN     none  
24148         2.0       NaN         NaN       NaN     none  
24149         3.0       

In [67]:
plac.head()

,#chr,start,end,count,expected,fdr,celltype,id,link,link_id
0,chr1,890000,895000,36,13.7267118749079,7.08770821896245e-05,iN,iN.14,chr1:890000-895000_chr1:900000-905000,1
1,chr1,900000,905000,36,13.7267118749079,7.08770821896245e-05,iN,iN.14,chr1:890000-895000_chr1:900000-905000,2
2,chr1,1000000,1005000,13,3.17293864858796,0.0005259819865644,iN,iN.259,chr1:1000000-1005000_chr1:1300000-1305000,1
3,chr1,1005000,1010000,13,3.19541920865153,0.0015132910675641,iN,iN.1938,chr1:1005000-1010000_chr1:1305000-1310000,1
4,chr1,1020000,1025000,17,3.87774128536577,2.41664334727065e-05,iN,iN.218,chr1:1020000-1025000_chr1:1320000-1325000,1


In [35]:
#enh 1530
plac.loc[(plac["start"]=='109380000') & (plac["#chr"]=='chr6')]

,#chr,start,end,count,expected,fdr,celltype,id,link,link_id
147608,chr6,109380000,109385000,12,3.82557551796205,0.003183063515626,eN,eN.9621,chr6:108560000-108565000_chr6:109380000-109385000,2
147609,chr6,109380000,109385000,17,3.44369076410255,1.65726015668907e-06,eN,eN.9644,chr6:108555000-108560000_chr6:109380000-109385000,2


In [ ]:
#enh 123
plac.loc[(plac["start"]=='109380000') & (plac["#chr"]=='chr6')]